In [104]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from collections import defaultdict
from itertools import combinations
from collections import deque, defaultdict
from itertools import chain
from itertools import islice

In [105]:
df = pd.read_csv("vk.csv", delimiter=',')

In [106]:
df.head()
df.shape

(14847753, 4)

## Число вершин

In [4]:
vertices = len(set([*df["u"].unique(), *df["v"].unique()]))
print(vertices)

3103653


## Число всех ребер

In [5]:
print(f'Число ребер в графе: {len(df["u"])}')

Число ребер в графе: 14847753


In [6]:
print(f'Теоретически возможное число ребер в данном графе: {vertices*(vertices-1)}')

Теоретически возможное число ребер в данном графе: 9632658840756


## Число уникальных ребер

In [7]:
# unique_edges = len(np.unique(df[['u', 'v', 't', 'h']].values, axis=0))
unique_edges = np.unique(df[['u', 'v']].values, axis=0)
print(f'Число уникальных ребер: {len(unique_edges)}')

Число уникальных ребер: 14847753


## Плотность графа
Для неориентированного простого графа плотность графа с числом вершин V  определяется как отношение числа его рёбер E к числу рёбер полного графа

In [49]:
density = 2*unique_edges/((vertices*(vertices-1)))
print(f'Плотность: {density}')

Плотность: [[1.03813497e-12 4.43113793e-07]
 [1.03813497e-12 9.89717601e-07]
 [1.03813497e-12 1.42484834e-06]
 ...
 [3.31587618e-06 3.31699031e-06]
 [3.31638590e-06 3.31811108e-06]
 [3.31805253e-06 3.31823565e-06]]


## Компоненты слабой связности

In [50]:
class Node:
    def __init__(self, key):
        self.key = key
        self.parent = self
        self.size = 1

class UnionFind(dict):
    def find(self, key):
        node = self.get(key, None)
        if node is None:
            node = self[key] = Node(key)
        else:
            while node.parent != node: 
                # walk up & perform path compression
                node.parent, node = node.parent.parent, node.parent
        return node

    def union(self, key_a, key_b):
        node_a = self.find(key_a)
        node_b = self.find(key_b)
        if node_a != node_b:  # disjoint? -> join!
            if node_a.size < node_b.size:
                node_a.parent = node_b
                node_b.size += node_a.size
            else:
                node_b.parent = node_a
                node_a.size += node_b.size

In [6]:
def find_components(line_iterator):
    forest = UnionFind()

    for line in line_iterator:
        forest.union(line[0], line[1])

    result = defaultdict(list)
    for key in forest.keys():
        root = forest.find(key)
        result[root.key].append(key)

    return list(result.values())

In [52]:
graph_edges = [edge for edge in df[["u","v"]].values]

In [53]:
graph_components = find_components(graph_edges)

In [54]:
print(f'Число компонент слабой связности: {len(graph_components)}')

max_component = max(graph_components, key=lambda i: len(i))
print(f'Число вершин в максимальной компоненте связности: {len(max_component)}')
print(f'Доля вершин в максимальной по мощности компоненте слабой связности: {len(max_component)/vertices}')

Число компонент слабой связности: 28175
Число вершин в максимальной компоненте связности: 3041820
Доля вершин в максимальной по мощности компоненте слабой связности: 0.9800773475643056


## Радиус и диаметр наибольшей компоненты слабой связности

Эксцентриситет вершины - расстояние от нее до самой удаленной.  
Диаметр графа - максимальное расстояние между любыми двумя вершинами, то есть наибольший эксцентриситет.  
Радиус графа - наименьший эксцентриситет.

In [107]:
def binary_search(lys, val):
    first = 0
    last = len(lys)-1
    index = -1
    while (first <= last) and (index == -1):
        mid = (first+last)//2
        if lys[mid] == val:
            index = mid
        else:
            if val<lys[mid]:
                last = mid -1
            else:
                first = mid +1
    if index == -1:
        return False
    return index

test = [1,2,3,4]
if binary_search(test,7):
    print(1)

In [32]:
# test = [edge for edge in df.values]
# print(test[:10])
diameter = 0
radius = float("inf")
# max_component_edges = [edge[2] for edge in df.values[:1000] if edge[0] and edge[1] in max_component]

random_edges_max_component = []

# while len(random_edges_max_component) != 500:
#     random_egde = random.choice(df.values)
#     if random_egde[0] in max_component and random_egde[1] in max_component:
#         random_edges_max_component.append(random_egde)
#     print(len(random_edges_max_component))
i=0
for value in df.values[30000:]:
#     if binary_search(max_component, value[0]) and binary_search(max_component, value[1]):
    if value[0] in max_component and value[1] in max_component:
        print(i)
        i+=1
        diameter = max(diameter, value[3])
        print(f"Diameter: {diameter}")
        print(f"Radius: {radius}")
        radius = min(radius, value[3])
        if i==500:
            break

print(random_edges_max_component)

print(f"Diameter: {diameter}")
print(f"Radius: {radius}")
# max_component_edges = df[(df['u'] in max_component) & (df['v'] in max_component)]
# print(max_component_edges)

0
Diameter: 8.0
Radius: inf
1
Diameter: 8.0
Radius: 8.0
2
Diameter: 8.0
Radius: 4.0
3
Diameter: 8.0
Radius: 4.0
4
Diameter: 8.0
Radius: 2.0
5
Diameter: 8.0
Radius: 2.0
6
Diameter: 8.0
Radius: 0.0
7
Diameter: 9.0
Radius: 0.0
8
Diameter: 9.0
Radius: 0.0
9
Diameter: 9.0
Radius: 0.0
10
Diameter: 9.0
Radius: 0.0
11
Diameter: 9.0
Radius: 0.0
12
Diameter: 9.0
Radius: 0.0
13
Diameter: 9.0
Radius: 0.0
14
Diameter: 9.0
Radius: 0.0
15
Diameter: 9.0
Radius: 0.0
16
Diameter: 9.0
Radius: 0.0
17
Diameter: 9.0
Radius: 0.0
18
Diameter: 9.0
Radius: 0.0
19
Diameter: 9.0
Radius: 0.0
20
Diameter: 9.0
Radius: 0.0
21
Diameter: 9.0
Radius: 0.0
22
Diameter: 9.0
Radius: 0.0
23
Diameter: 9.0
Radius: 0.0
24
Diameter: 9.0
Radius: 0.0
25
Diameter: 9.0
Radius: 0.0
26
Diameter: 9.0
Radius: 0.0
27
Diameter: 9.0
Radius: 0.0
28
Diameter: 9.0
Radius: 0.0
29
Diameter: 9.0
Radius: 0.0
30
Diameter: 9.0
Radius: 0.0
31
Diameter: 9.0
Radius: 0.0
32
Diameter: 9.0
Radius: 0.0
33
Diameter: 9.0
Radius: 0.0
34
Diameter: 9.0
Radius:

KeyboardInterrupt: 

## Загрузка графов

In [113]:
def load_graph(path):
    print("Loading...{}".format(path))
    graph = {}  # graph : Dictionary = { node : { node1, node2, node3} }
    f = open(path, "r")
    i = 0
    while True:
        line = f.readline().strip(',')
        if line.startswith("#"):
            continue

        if not line:
            break
        
        node_from, node_to = map(int, line.split(',')[:2])

        # since graph is undirected
        if node_from not in graph:
            graph[node_from] = set()
        if node_to not in graph:
            graph[node_to] = set()

        graph[node_from].add(node_to)
        graph[node_to].add(node_from)

    return graph


def load_graph_by_whitespace(path):
    print("Loading...{}".format(path))
    graph = {}  # graph : Dictionary = { node : { node1, node2, node3} }
    f = open(path, "r")
    f.readline()
    while True:
        line = f.readline().strip()
        if not line:
            break
        if line.startswith("#"):
            continue
        node_from, node_to = map(int, line.split())

        # since graph is undirected
        if node_from not in graph:
            graph[node_from] = set()
        if node_to not in graph:
            graph[node_to] = set()

        graph[node_from].add(node_to)
        graph[node_to].add(node_from)

    return graph

In [124]:
df_test = pd.read_csv("test.txt", delimiter='\t')
graph_nodes_degrees_test = graph_nodes_degrees_test = load_graph_by_whitespace("/Users/antonkondrahin/PycharmProjects/FiniteGraphs/test.txt")
print(print_cliques(df_test[["u","v"]].values, 3, graph_nodes_degrees_test))

Loading.../Users/antonkondrahin/PycharmProjects/FiniteGraphs/test.txt
[{1, 2, 3}, {1, 4, 5}, {10, 11, 13}]


In [ ]:
df_astro = pd.read_csv("astro.txt", delimiter='\t')
graph_nodes_degrees_astro = load_graph_by_whitespace("/Users/antonkondrahin/PycharmProjects/FiniteGraphs/astro.txt")
print_cliques(df_astro[["u","v"]].values, 3, graph_nodes_degrees_astro)

Loading.../Users/antonkondrahin/PycharmProjects/FiniteGraphs/astro.txt


## Локальный кластерный коэффициент вершины

### Число треугольников (клик размера 3)

In [174]:
def enumerate_all_cliques(G):
    """Returns all cliques in an undirected graph.

    This function returns an iterator over cliques, each of which is a
    list of nodes. The iteration is ordered by cardinality of the
    cliques: first all cliques of size one, then all cliques of size
    two, etc.

    Returns
    -------
    iterator
        An iterator over cliques, each of which is a list of nodes in
        `G`. The cliques are ordered according to size.

    Notes
    -----
    To obtain a list of all cliques, use
    `list(enumerate_all_cliques(G))`. However, be aware that in the
    worst-case, the length of this list can be exponential in the number
    of nodes in the graph (for example, when the graph is the complete
    graph). This function avoids storing all cliques in memory by only
    keeping current candidate node lists in memory during its search.
    """
    index = {}
    nbrs = {}
    for u in G:
        index[u] = len(index)
        # Neighbors of u that appear after u in the iteration order of G.
        nbrs[u] = {v for v in G[u] if v not in index}

    queue = deque(([u], sorted(nbrs[u], key=index.__getitem__)) for u in G)
    # Loop invariants:
    # 1. len(base) is nondecreasing.
    # 2. (base + cnbrs) is sorted with respect to the iteration order of G.
    # 3. cnbrs is a set of common neighbors of nodes in base.
    while queue:
        base, cnbrs = map(list, queue.popleft())
        yield base
        for i, u in enumerate(cnbrs):
            # Use generators to reduce memory consumption.
            queue.append(
                (
                    chain(base, [u]),
                    filter(nbrs[u].__contains__, islice(cnbrs, i + 1, None)),
                )
            )
            
def count_k_cliques(G, k, vertice_clique):
    k_cliques_count = 0
    for clique in enumerate_all_cliques(G): 
        if len(clique) > k: 
            break
        elif len(clique) == k:
            for vertice in clique:
                vertice_clique[vertice].add(frozenset(clique))
            k_cliques_count += 1
    return k_cliques_count, vertice_clique

In [ ]:
vertice_clique = defaultdict(set)
graph_cliques_number, graph_cliques_by_node = count_k_cliques(graph_nodes_degrees_astro, 3, vertice_clique)
print(graph_cliques_number)

In [ ]:
vertice_clique_test = defaultdict(set)
graph_cliques_number_test, graph_cliques_by_node_test = count_k_cliques(graph_nodes_degrees_test, 3, vertice_clique_test)
print(graph_cliques_number_test)
print(graph_cliques_by_node_test)

### Локальные коэффициенты

Если кратко, то локальный коэффициент - это отношение числа треугольников, где есть искомая вершина к степени вершины. Подробнее туть
https://translated.turbopages.org/proxy_u/en-ru.ru.7d7cd3c3-627e6c61-479fa8f8-74722d776562/https/en.wikipedia.org/wiki/Clustering_coefficient

In [195]:
local_coeffs_test = defaultdict(int)
for node, cliques in graph_cliques_by_node_test.items():
    node_degree = len(graph_nodes_degrees_test.get(node, []))
    if node_degree>=2:
        local_coeffs_test[node] = 2*len(cliques)/(node_degree*(node_degree-1))
    else:
        local_coeffs_test[node] = 0

In [196]:
local_coeffs = defaultdict(int)
for node, cliques in graph_cliques_by_node.items():
    node_degree = len(graph_nodes_degrees_astro.get(node, []))
    if node_degree>=2:
        local_coeffs[node] = 2*len(cliques)/(node_degree*(node_degree-1))
    else:
        local_coeffs[node] = 0

In [197]:
print(f'Локальные коэффициенты для вершин графа: {local_coeffs}')

Локальные коэффициенты для вершин графа: defaultdict(<class 'int'>, {84424: 0.07567567567567568, 276: 0.11612903225806452, 20113: 0.1164021164021164, 53681: 0.3484848484848485, 58458: 0.6666666666666666, 63552: 0.16164762893734858, 66200: 0.167816091954023, 97101: 0.13852813852813853, 130825: 0.1111111111111111, 1662: 0.24141749723145073, 33040: 0.0838703793774319, 76259: 0.18963922294172064, 5089: 1.0, 6058: 0.2292134831460674, 25452: 0.15367249858905105, 39521: 0.2012119851868477, 45009: 0.16401788785689714, 77098: 0.23114754098360657, 83560: 0.13243118204950266, 132445: 0.14411288457089985, 6229: 0.12427409988385599, 61571: 0.16010854816824965, 69839: 0.053410893707033315, 92387: 0.20915032679738563, 106611: 0.1583860283047275, 107829: 0.24079485680888368, 10639: 0.11878893234825438, 32432: 0.20317460317460317, 39238: 0.24242424242424243, 50220: 0.19047619047619047, 73543: 0.14692202462380302, 91060: 0.15324675324675324, 16442: 0.31494252873563217, 19325: 0.13084702907711757, 47968:

### Средний коэффициент

In [204]:
average_coeff = sum(local_coeffs.values())/len(graph_nodes_degrees_astro)
print(f'Средний кластерный коэффициент сети: {average_coeff}')

Средний кластерный коэффициент сети: 0.6303341692292374


### Глобальный коэффициент

In [200]:
def С(n, k):
    if 0 <= k <= n:
        nn = 1
        kk = 1
        for t in range(1, min(k, n - k) + 1):
            nn *= n
            kk *= t
            n -= 1
        return nn // kk
    else:
        return 0

In [206]:
numerator = 0
denominator = 0
for node, local_coeff in local_coeffs.items():
    node_degree = len(graph_nodes_degrees_astro.get(node, []))
    c_n_k = С(node_degree, 2)
    denominator += c_n_k
    numerator += c_n_k*local_coeff

global_coeff = numerator/denominator

In [207]:
print(f'Глобальный кластерный коэффициент: {global_coeff}')

Глобальный кластерный коэффициент: 0.3179284915494925
